In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Making things easy with SimpleTransformers

In [ ]:
!pip install git+git://github.com/AndLen/simpletransformers.git --quiet

In [ ]:
import os
import gc
from simpletransformers.classification import (ClassificationModel, ClassificationArgs)
import sklearn
from sklearn.model_selection import train_test_split
import torch
import plotly.express as px

In [ ]:
#this is for silencing some tokenizer warnings
os.environ["TOKENIZERS_PARALLELISM"] = "false"
RANDOM_STATE = 42

## Reading our big-boy data

In [ ]:
test = pd.read_csv("../input/contradictory-my-dear-watson/test.csv")
training = pd.read_csv("../input/contradictory-my-dear-watson/train.csv")

We are renaming columns to Simple Transformers specifications

In [ ]:
training_df = training[["premise", "hypothesis", "label"]]
training_df.columns = ["text_a", "text_b", "labels"]

In [ ]:
final_test_df = test[["premise", "hypothesis"]]
final_test_df.columns = ["text_a", "text_b"]

Some langauages distributions, quite unbalanced

In [ ]:
fig = px.histogram(training, x="language")
fig.show()

On the other side, labels are quite balanced

In [ ]:
training_df["labels"].value_counts()

## train test eval split, 70:15:15 proportions

We split the datasate using stratification when testing

In [ ]:
#train_df, test_df = train_test_split(training_df, test_size=0.30, random_state=RANDOM_STATE, stratify=training_df["labels"])
#eval_df, test_df = train_test_split(test_df, test_size=0.50, random_state=RANDOM_STATE, stratify=test_df["labels"])

In [ ]:
shuffled_training = training_df.sample(frac=1, random_state=RANDOM_STATE, ignore_index=True)#.reset_index(drop=True)

In [ ]:
shuffled_training.shape #train_df.shape ,test_df.shape, eval_df.shape

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# Create a ClassificationModel
model_args = ClassificationArgs(num_train_epochs=3, # One big-boy epoch is enough for this chonky boy
                                overwrite_output_dir=True)
model_args.manual_seed = RANDOM_STATE
model_args.best_model_dir = "/kaggle/working/best_model"
model_args.output_dir = "/kaggle/temp/output"

#model_args.reprocess_input_data = True
#model_args.evaluate_during_training = True
#model_args.evaluate_during_training_verbose = True
model_args.train_batch_size = 24
model_args.eval_batch_size = 24

#model_args.early_stopping_metric = "mcc"
#model_args.early_stopping_metric_minimize = False
#model_args.save_eval_checkpoints = False
#model_args.use_early_stopping = True
#model_args.early_stopping_consider_epochs = True
#model_args.early_stopping_patience = 3
model_args.learning_rate = 1e-6
model_args.manual_seed = RANDOM_STATE

model = ClassificationModel(model_type='xlmroberta', 
                            model_name='joeddav/xlm-roberta-large-xnli', 
                            args = model_args, 
                            num_labels = 3)

In [ ]:
model.train_model(shuffled_training)
                  #eval_df=eval_df)

1ep = (758, 0.5433722181063844)
{'mcc': 0.9258332483255106,
 'acc': 0.9505775577557756,
 'eval_loss': 0.1682783056610807}

2ep = (1516, 0.3996097231111772)
{'mcc': 0.9530729053357176,
 'acc': 0.9687293729372938,
 'eval_loss': 0.12447924599405329}

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(shuffled_training, 
                                                            acc=sklearn.metrics.accuracy_score)

In [ ]:
result

## Predicting on unlabelled test set and saving

In [ ]:
final_pred_texts = []
for row in final_test_df.itertuples(index = False):
    final_pred_texts.append(list(row))

In [ ]:
predictions, raw_outputs = model.predict(final_pred_texts)

In [ ]:
mypreds = pd.DataFrame(test["id"])
mypreds["prediction"] = predictions

In [ ]:
mypreds.to_csv("submission.csv", index=False)